# UKy Flowsheet with Costing

The purpose of this tutorial is to demonstrate defining, adding, and solving costing for the University of Kentucky flowsheet. It is assumed that the user is already familiar with the flowsheet, which is introduced in detail in the [UKy Flowsheet Tutorial](uky_flowsheet-solution.ipynb). The user should have an understanding of the basic features and structure of the costing framework in PrOMMiS, which is introduced in detail in the [Basic Costing Features](costing_basic_features-solution.ipynb) tutorial.

## Problem Statement

This tutorial will show how to add costing to the West Kentucky No.13 Coal Refuse flowsheet, referred to in this tutorial as the University of Kentucky (UKy) flowsheet. As is the case for the flowsheet model, the inputs for the costing are case study-specific, so the flowsheet is not guaranteed to solve if the values are significantly altered.

![uky_flowsheet.png](./uky_flowsheet.png)

The tutorial will take users through the following:

1. Importing the required tools from PrOMMiS and related repositories
2. Importing and building the UKy flowsheet
3. Adding costing to the UKy flowsheet
4. Initializing, solving, and displaying key cost results

Useful Links:
* Public GitHub Repository: https://github.com/prommis/prommis/tree/main
* REE Costing Module Code: https://github.com/prommis/prommis/blob/main/src/prommis/uky/costing/ree_plant_capcost.py

# 1 Import the necessary tools

First, import the required Python, Pyomo, IDAES, and PrOMMiS packages. These will be implemented at various stages of the demonstration.

For installation instructions, please refer to the public GitHub repository linked above.

In [1]:
# import pytest
import pytest

# Pyomo packages
from pyomo.environ import (
    assert_optimal_termination,
    ConcreteModel,
    SolverFactory,
    Suffix,
    TransformationFactory,
    Constraint,
    Var,
    Param,
    Expression,
    units as pyunits,
    check_optimal_termination,
    value,
)

# IDAES packages
from idaes.core import FlowsheetBlock, UnitModelBlock, UnitModelCostingBlock
from idaes.core.solvers import get_solver
from idaes.core.util.model_diagnostics import DiagnosticsToolbox
from idaes.core.util.model_statistics import degrees_of_freedom

# PrOMMiS packages
from prommis.uky.uky_flowsheet import (
    add_result_expressions,
    build,
    set_operating_conditions,
    set_scaling,
    initialize_system,
    solve_system,
    fix_organic_recycle,
    display_results,
)
from prommis.uky.costing.ree_plant_capcost import QGESSCosting, QGESSCostingData
from prommis.uky.costing.costing_dictionaries import load_REE_costing_dictionary

# 2 Build the UKy Flowsheet

## 2.1 Process Description
The University of Kentucky (UKy) Flowsheet in PrOMMiS simulates a West Kentucky No.  13 Coal Refuse processing plant in which rare earth element (REE) components are recovered via a series of mechanical and chemical treatment processes. A diagram of the process is shown below:

![uky_flowsheet.png](./uky_flowsheet.png)

## 2.2 Build the Flowsheet
In this demonstration, we will call the flowsheet methods to build the flowsheet and will not show the model structure in detail. For more information, see the [UKy Flowsheet Tutorial](uky_flowsheet-solution.ipynb).

The flowsheet methods are called below:

In [2]:
# Call the flowsheet methods to build and connect the unit operations
m = build()

set_operating_conditions(m)

set_scaling(m)

add_result_expressions(m)

if degrees_of_freedom(m) != 0:
    raise AssertionError(
        "The degrees of freedom are not equal to 0."
        "Check that the expected variables are fixed and unfixed."
        "For more guidance, run assert_no_structural_warnings from the IDAES DiagnosticToolbox "
    )

initialize_system(m)

solve_system(m)

# Fixes the volumetric flow rate of the organic recycle streams and unfixes the flow of the make-up streams
# We want to be able to adjust the total recycle flow rate, not just the make-up portion of it
fix_organic_recycle(m)

results = solve_system(m)

if not check_optimal_termination(results):
    raise RuntimeError(
        "Solver failed to terminate with an optimal solution. Please check the solver logs for more details"
    )

# Adjust inputs to commercial scale
scaleup_factor = value(
    pyunits.convert(
        # UKy solid feed rate to leaching circuit
        495 * pyunits.ton / pyunits.hr,
        to_units=pyunits.kg / pyunits.hr,
    )
    / (m.fs.leach_solid_feed.flow_mass[0])
)

for var in [
    m.fs.leach_liquid_feed.flow_vol,
    m.fs.leach_solid_feed.flow_mass,
    m.fs.leach.volume,
    m.fs.rougher_org_make_up.flow_vol,
    m.fs.acid_feed1.flow_vol,
    m.fs.acid_feed2.flow_vol,
    m.fs.acid_feed3.flow_vol,
    m.fs.cleaner_org_make_up.flow_vol,
    m.fs.roaster.gas_inlet.flow_mol,
]:
    for k in var.keys():
        var[k].fix(
            value(var[k]) * 1
        )  # TODO try reverting to use scaleup_factor once UKy flowsheet is fixed

results = get_solver("ipopt_v2").solve(m, tee=True)

display_results(m)

Scaling fs.leach
Scaling fs.sl_sep1
Scaling fs.scrubber_HCl_leach_translator
Scaling fs.leach_mixer
Scaling fs.leach_liquid_feed
Scaling fs.leach_solid_feed
Scaling fs.leach_filter_cake
Scaling fs.leach_filter_cake_liquid
Scaling fs.rougher_org_make_up
Scaling fs.solex_rougher_load
Scaling fs.acid_feed1
Scaling fs.solex_rougher_scrub
Scaling fs.acid_feed2
Scaling fs.solex_rougher_strip
Scaling fs.rougher_sep
Scaling fs.rougher_mixer
Scaling fs.load_sep
Scaling fs.scrub_sep
Scaling fs.rougher_organic_purge
Scaling fs.solex_cleaner_load
Scaling fs.solex_cleaner_strip
Scaling fs.cleaner_org_make_up
Scaling fs.cleaner_mixer
Scaling fs.cleaner_sep
Scaling fs.cleaner_HCl_leach_translator
Scaling fs.leach_sx_mixer
Scaling fs.acid_feed3
Scaling fs.cleaner_organic_purge
Scaling fs.precipitator
Scaling fs.sl_sep2
Scaling fs.precip_sep
Scaling fs.precip_sx_mixer
Scaling fs.precip_purge
Scaling fs.roaster
Scaling fs.leaching_sol_feed_expanded
Scaling fs.leaching_liq_feed_expanded
Scaling fs.leachi

# 3 Add Costing

To begin, we need to create a flowsheet-level costing block. This serves as a central location for plant-wide costing, as well as a reference to let equipment costing import the costing library and equations from the specified source. We also need to define the cost year; Pyomo supports conversion of cost years from 1990-2023 and setting this string will automatically convert flowsheet cost results to the specified cost year.

In this demonstration, we will use the `QGESSCosting()` method imported from the module `ree_plant_capcost.py` which support the economic formulation used in the UKy study. The Quality Guidelines for Energy Systems Studies (QGESS) form the backbone of the costing framework, incorporating economies of scale for equipment costing and assumptions for operating, maintenance, installation, and overnight cost estimation based on prior knowledge of industrial processes. The QGESS methods and implementation are introduced for PrOMMiS modeling in the [Basic Costing Features](costing_basic_features-solution.ipynb) and more generally in the [IDAES Power Plant Costing](https://idaes-pse.readthedocs.io/en/stable/reference_guides/model_libraries/power_generation/costing/power_plant_costing_netl.html) documentation.

Now, let's add the flowsheet-level costing block and set the cost year:

In [3]:
m.fs.costing = QGESSCosting()
CE_index_year = "2023"

## 3.1 Build capital costing for a unit model

Next, we need to build and attach capital costing equations for each unit model block via the IDAES `UnitModelCostingBlock()` method. Costing may be attached to an imported, fully-defined unit model, or a dummy block such as Pyomo's `Block()` or IDAES's `UnitModelBlock()`. The latter option is useful to cost balance-of-plant equipment that is not explicitly modeled in the flowsheet.

To begin, let's add costing for the leaching tanks:

In [4]:
# 4.2 is UKy Leaching - Polyethylene Tanks
L_pe_tanks_accounts = ["4.2"]
m.fs.leach.costing = UnitModelCostingBlock(
    flowsheet_costing_block=m.fs.costing,
    costing_method=QGESSCostingData.get_REE_costing,
    costing_method_arguments={
        "cost_accounts": L_pe_tanks_accounts,
        "scaled_param": m.fs.leach.volume[0, 1],
        "source": 1,
        "n_equip": 3,
        "scale_down_parallel_equip": False,
        "CE_index_year": CE_index_year,
    },
)

The `UnitModelCostingBlock` takes several arguments:

* `flowsheet_costing_block` - a flowsheet-level costing block to attach the cost model to, in this case the one we created.
* `costing_method` - the method to use for equipment cost calculations, which exists in the `QGESSCostingData` class.

There are a few arguments for the `QGESSCosting` method:
* `cost_accounts` - the list of reference cost accounts from the PrOMMiS library to use; we are only using one here, but the method supports multiple cost accounts if they scale with the same scaling parameter, use the same reference source, and have the same number of parallel units.
* `scaled_param` - the variable used to calculate the scaled cost per the following power law for economies of scale:    

$ scaledCost = referenceCost * (\frac{scaledParam}{referenceParam})^{EXP}$    

where the values of `referencecost`, `referenceparam`, and `EXP` are sourced from the PrOMMiS cost account library.

* `source` - the library account list to draw from; PrOMMiS currently supports two sources: a large number of equipment accounts from the UKy flowsheet ("1") and additional accounts for magnet recycling ("2").
* `n_equip` - the number of parallel equipment, for example we have 3 leach tanks in parallel
* `scale_down_parallel_equip` - whether the scaling parameter is the capacity of the entire train ("True") or each individual unit in the train ("False").

For example, if set to "False" the scaled parameter (flowsheet variable for tank volume) is taken as the volume of each individual leach tank:

$ trainCost = nEquip * referenceCost * (\frac{scaledParam}{referenceParam})^{EXP} = nEquip * scaledCost$

If set to "True", the volume is taken as the total volume of all tanks in the train, and the individual volumes are scaled down:

$ trainCost = nEquip * referenceCost * (\frac{scaledParam}{nEquip * referenceParam})^{EXP} = nEquip * scaledCost * (\frac{1}{nEquip})^{EXP} = nEquip^{1-EXP} * scaledCost$

Due to economies of scale (cost per capacity becomes cheaper as capacity increases), the exponents for most cost accounts are less than 1. This option is useful when equipment is not constrained by a discrete set of available capacities, and the total capacity of a train is more readily available than the size of each individual unit.

* `CE_index_year` - the basis year for flowsheet cost calculations, which we set as "2023" earlier.

# 3.2 Build capital costing for the rest of the flowsheet
Let's do this for all other equipment. Each equipment requires its own costing block, and the syntax is exactly the same as how we costed the leach tank. 

However, many of the balance-of-plant equipment (pumps, tanks, filters) either do not have explicitly unit models in the flowsheet, or the process parameter to estimate the scaled capital cost is not known. These unit operations need to be included in the cost, so blocks will be created for them. For example, additional cost components for leaching are attached to `UnitModelBlock()` objects, since a unit model can only have one costing block attached. An empirical scaling is applied using the flowrate into the leaching, rougher solvent extraction cleaner solvent extraction, and precipitation sections to estimate non-flow parameters using data from the [University of Kentucky report](https://doi.org/10.2172/1569277).

The rest of the equipment costing is added below:

In [5]:
# Define reference values for empirical scaling to estimate balance of
# Plant unit operation process parameters
# scaled_parameter = reference_parameter * (scaled_basis_flow/reference_basis_flow)

# Reference values from UKy study - Table 4-7 p. 351
REE_costing_params = load_REE_costing_dictionary()
reference_basis_flow = {
    "leach_sol_flow_mass": 495 * pyunits.ton / pyunits.hr,  # p. 273, 351
    "rougher_solex_aqueous_flow_vol": 23131 * pyunits.L / pyunits.min,
    "cleaner_solex_aqueous_flow_vol": 925 * pyunits.L / pyunits.min,
    "precipitator_solex_aqueous_flow_vol": 231 * pyunits.L / pyunits.min,
}

# Leaching costs
# 4.3 is UKy Leaching - Tank Mixer
L_tank_mixer_accounts = ["4.3"]
m.fs.leach_mixer.power = Var(initialize=4.74, units=pyunits.hp, bounds=(0, None))


@m.fs.leach_mixer.Constraint(L_tank_mixer_accounts)
def power_scaling_constraint(c, k):
    return m.fs.leach_mixer.power == pyunits.convert(
        REE_costing_params["1"][k]["RP Value"]
        * pyunits.hp
        * (
            m.fs.leach_solid_feed.flow_mass[0]
            / reference_basis_flow["leach_sol_flow_mass"]
        ),
        to_units=pyunits.hp,
    )


m.fs.leach_mixer.costing = UnitModelCostingBlock(
    flowsheet_costing_block=m.fs.costing,
    costing_method=QGESSCostingData.get_REE_costing,
    costing_method_arguments={
        "cost_accounts": L_tank_mixer_accounts,
        "scaled_param": m.fs.leach_mixer.power,
        "source": 1,
        "n_equip": 3,
        "scale_down_parallel_equip": False,
        "CE_index_year": CE_index_year,
    },
)

# 4.4 is UKy Leaching - Process Pump
L_pump_accounts = ["4.4"]
m.fs.leach_pump = UnitModelBlock()
m.fs.leach_pump.costing = UnitModelCostingBlock(
    flowsheet_costing_block=m.fs.costing,
    costing_method=QGESSCostingData.get_REE_costing,
    costing_method_arguments={
        "cost_accounts": L_pump_accounts,
        "scaled_param": m.fs.leach_liquid_feed.flow_vol[0],
        "source": 1,
        "n_equip": 3,
        "scale_down_parallel_equip": False,
        "CE_index_year": CE_index_year,
    },
)

# 4.5 is UKy Leaching - Thickener
L_thickener_accounts = ["4.5"]
m.fs.leach_sx_mixer.area = Var(initialize=225.90, units=pyunits.ft**2, bounds=(0, None))


@m.fs.leach_sx_mixer.Constraint(L_thickener_accounts)
def area_scaling_constraint(c, k):
    return m.fs.leach_sx_mixer.area == pyunits.convert(
        REE_costing_params["1"][k]["RP Value"]
        * pyunits.ft**2
        * (
            m.fs.leach_solid_feed.flow_mass[0]
            / reference_basis_flow["leach_sol_flow_mass"]
        ),
        to_units=pyunits.ft**2,
    )


m.fs.leach_sx_mixer.costing = UnitModelCostingBlock(
    flowsheet_costing_block=m.fs.costing,
    costing_method=QGESSCostingData.get_REE_costing,
    costing_method_arguments={
        "cost_accounts": L_thickener_accounts,
        "scaled_param": m.fs.leach_sx_mixer.area,
        "source": 1,
        "n_equip": 1,
        "scale_down_parallel_equip": False,
        "CE_index_year": CE_index_year,
    },
)

# 4.6 is UKy Leaching - Solid Waste Filter Press
L_filter_press_accounts = ["4.6"]
m.fs.sl_sep1.volume = Var(initialize=36.00, units=pyunits.ft**3, bounds=(0, None))


@m.fs.sl_sep1.Constraint(L_filter_press_accounts)
def volume_scaling_constraint(c, k):
    return m.fs.sl_sep1.volume == pyunits.convert(
        REE_costing_params["1"][k]["RP Value"]
        * pyunits.ft**3
        * (
            m.fs.leach_solid_feed.flow_mass[0]
            / reference_basis_flow["leach_sol_flow_mass"]
        ),
        to_units=pyunits.ft**3,
    )


m.fs.sl_sep1.costing = UnitModelCostingBlock(
    flowsheet_costing_block=m.fs.costing,
    costing_method=QGESSCostingData.get_REE_costing,
    costing_method_arguments={
        "cost_accounts": L_filter_press_accounts,
        "scaled_param": m.fs.sl_sep1.volume,
        "source": 1,
        "n_equip": 1,
        "scale_down_parallel_equip": False,
        "CE_index_year": CE_index_year,
    },
)

# 4.8 is UKy Leaching - Solution Heater
L_solution_heater_accounts = ["4.8"]
m.fs.leach_solution_heater = UnitModelBlock()
m.fs.leach_solution_heater.duty = Var(
    initialize=0.24, units=pyunits.MBTU / pyunits.hr, bounds=(0, None)
)


@m.fs.leach_solution_heater.Constraint(L_solution_heater_accounts)
def duty_scaling_constraint(c, k):
    return m.fs.leach_solution_heater.duty == pyunits.convert(
        REE_costing_params["1"][k]["RP Value"]
        * pyunits.MBTU
        / pyunits.hr
        * (
            m.fs.leach_solid_feed.flow_mass[0]
            / reference_basis_flow["leach_sol_flow_mass"]
        ),
        to_units=pyunits.MBTU / pyunits.hr,
    )


m.fs.leach_solution_heater.costing = UnitModelCostingBlock(
    flowsheet_costing_block=m.fs.costing,
    costing_method=QGESSCostingData.get_REE_costing,
    costing_method_arguments={
        "cost_accounts": L_solution_heater_accounts,
        "scaled_param": m.fs.leach_solution_heater.duty,
        "source": 1,
        "n_equip": 1,
        "scale_down_parallel_equip": False,
        "CE_index_year": CE_index_year,
    },
)

# Solvent extraction costs
# 5.1 is UKy Rougher Solvent Extraction - Polyethylene Tanks
RSX_pe_tanks_accounts = ["5.1"]
m.fs.rougher_solex_tank = UnitModelBlock()
m.fs.rougher_solex_tank.volume = Var(
    initialize=35.136, units=pyunits.gal, bounds=(0, None)
)


@m.fs.rougher_solex_tank.Constraint(RSX_pe_tanks_accounts)
def volume_scaling_constraint(c, k):
    return m.fs.rougher_solex_tank.volume == pyunits.convert(
        REE_costing_params["1"][k]["RP Value"]
        * pyunits.gal
        * (
            m.fs.solex_rougher_load.mscontactor.aqueous_inlet.flow_vol[0]
            / reference_basis_flow["rougher_solex_aqueous_flow_vol"]
        ),
        to_units=pyunits.gal,
    )


m.fs.rougher_solex_tank.costing = UnitModelCostingBlock(
    flowsheet_costing_block=m.fs.costing,
    costing_method=QGESSCostingData.get_REE_costing,
    costing_method_arguments={
        "cost_accounts": RSX_pe_tanks_accounts,
        "scaled_param": m.fs.rougher_solex_tank.volume,
        "source": 1,
        "n_equip": 6,
        "scale_down_parallel_equip": False,
        "CE_index_year": CE_index_year,
    },
)

# 5.2 is UKy Rougher Solvent Extraction - Tank Mixer
RSX_tank_mixer_accounts = ["5.2"]
m.fs.rougher_mixer.power = Var(initialize=2.0, units=pyunits.hp, bounds=(0, None))


@m.fs.rougher_mixer.Constraint(RSX_tank_mixer_accounts)
def power_scaling_constraint(c, k):
    return m.fs.rougher_mixer.power == pyunits.convert(
        REE_costing_params["1"][k]["RP Value"]
        * pyunits.hp
        * (
            m.fs.solex_rougher_load.mscontactor.aqueous_inlet.flow_vol[0]
            / reference_basis_flow["rougher_solex_aqueous_flow_vol"]
        ),
        to_units=pyunits.hp,
    )


m.fs.rougher_mixer.costing = UnitModelCostingBlock(
    flowsheet_costing_block=m.fs.costing,
    costing_method=QGESSCostingData.get_REE_costing,
    costing_method_arguments={
        "cost_accounts": RSX_tank_mixer_accounts,
        "scaled_param": m.fs.rougher_mixer.power,
        "source": 1,
        "n_equip": 2,
        "scale_down_parallel_equip": False,
        "CE_index_year": CE_index_year,
    },
)

# 5.3 is UKy Rougher Solvent Extraction - Process Pump
RSX_pump_accounts = ["5.3"]
m.fs.rougher_pump = UnitModelBlock()
m.fs.rougher_pump.costing = UnitModelCostingBlock(
    flowsheet_costing_block=m.fs.costing,
    costing_method=QGESSCostingData.get_REE_costing,
    costing_method_arguments={
        "cost_accounts": RSX_pump_accounts,
        "scaled_param": m.fs.solex_rougher_load.mscontactor.aqueous_inlet.flow_vol[0],
        "source": 1,
        "n_equip": 1,
        "scale_down_parallel_equip": False,
        "CE_index_year": CE_index_year,
    },
)

# 5.4 is UKy Rougher Solvent Extraction - Mixer Settler
RSX_mixer_settler_accounts = ["5.4"]
m.fs.rougher_solex_settler = UnitModelBlock()
m.fs.rougher_solex_settler.volume = Var(
    initialize=61.107, units=pyunits.gal, bounds=(0, None)
)


@m.fs.rougher_solex_settler.Constraint(RSX_mixer_settler_accounts)
def volume_scaling_constraint(c, k):
    return m.fs.rougher_solex_settler.volume == pyunits.convert(
        REE_costing_params["1"][k]["RP Value"]
        * pyunits.gal
        * (
            m.fs.solex_rougher_load.mscontactor.aqueous_inlet.flow_vol[0]
            / reference_basis_flow["rougher_solex_aqueous_flow_vol"]
        ),
        to_units=pyunits.gal,
    )


m.fs.rougher_solex_settler.costing = UnitModelCostingBlock(
    flowsheet_costing_block=m.fs.costing,
    costing_method=QGESSCostingData.get_REE_costing,
    costing_method_arguments={
        "cost_accounts": RSX_mixer_settler_accounts,
        "scaled_param": m.fs.rougher_solex_settler.volume,
        "source": 1,
        "n_equip": 6,
        "scale_down_parallel_equip": False,
        "CE_index_year": CE_index_year,
    },
)

# 6.1 is UKy Cleaner Solvent Extraction - Polyethylene Tanks
CSX_pe_tanks_accounts = ["6.1"]
m.fs.cleaner_solex_tank = UnitModelBlock()
m.fs.cleaner_solex_tank.volume = Var(
    initialize=14.05, units=pyunits.gal, bounds=(0, None)
)


@m.fs.cleaner_solex_tank.Constraint(CSX_pe_tanks_accounts)
def volume_scaling_constraint(c, k):
    return m.fs.cleaner_solex_tank.volume == pyunits.convert(
        REE_costing_params["1"][k]["RP Value"]
        * pyunits.gal
        * (
            m.fs.solex_cleaner_load.mscontactor.aqueous_inlet.flow_vol[0]
            / reference_basis_flow["cleaner_solex_aqueous_flow_vol"]
        ),
        to_units=pyunits.gal,
    )


m.fs.cleaner_solex_tank.costing = UnitModelCostingBlock(
    flowsheet_costing_block=m.fs.costing,
    costing_method=QGESSCostingData.get_REE_costing,
    costing_method_arguments={
        "cost_accounts": CSX_pe_tanks_accounts,
        "scaled_param": m.fs.cleaner_solex_tank.volume,
        "source": 1,
        "n_equip": 5,
        "scale_down_parallel_equip": False,
        "CE_index_year": CE_index_year,
    },
)

# 6.2 is UKy Cleaner Solvent Extraction - Tank Mixer
CSX_tank_mixer_accounts = ["6.2"]
m.fs.cleaner_mixer.power = Var(initialize=0.08, units=pyunits.hp, bounds=(0, None))


@m.fs.cleaner_mixer.Constraint(CSX_tank_mixer_accounts)
def power_scaling_constraint(c, k):
    return m.fs.cleaner_mixer.power == pyunits.convert(
        REE_costing_params["1"][k]["RP Value"]
        * pyunits.hp
        * (
            m.fs.solex_cleaner_load.mscontactor.aqueous_inlet.flow_vol[0]
            / reference_basis_flow["cleaner_solex_aqueous_flow_vol"]
        ),
        to_units=pyunits.hp,
    )


m.fs.cleaner_mixer.costing = UnitModelCostingBlock(
    flowsheet_costing_block=m.fs.costing,
    costing_method=QGESSCostingData.get_REE_costing,
    costing_method_arguments={
        "cost_accounts": CSX_tank_mixer_accounts,
        "scaled_param": m.fs.cleaner_mixer.power,
        "source": 1,
        "n_equip": 2,
        "scale_down_parallel_equip": False,
        "CE_index_year": CE_index_year,
    },
)

# 6.3 is UKy Cleaner Solvent Extraction - Process Pump
CSX_pump_accounts = ["6.3"]
m.fs.cleaner_pump = UnitModelBlock()
m.fs.cleaner_pump.costing = UnitModelCostingBlock(
    flowsheet_costing_block=m.fs.costing,
    costing_method=QGESSCostingData.get_REE_costing,
    costing_method_arguments={
        "cost_accounts": CSX_pump_accounts,
        "scaled_param": m.fs.solex_cleaner_load.mscontactor.aqueous_inlet.flow_vol[0],
        "source": 1,
        "n_equip": 3,
        "scale_down_parallel_equip": False,
        "CE_index_year": CE_index_year,
    },
)

# 6.4 is UKy Cleaner Solvent Extraction - Mixer Settler
CSX_mixer_settler_accounts = ["6.4"]
m.fs.cleaner_solex_settler = UnitModelBlock()
m.fs.cleaner_solex_settler.volume = Var(
    initialize=24.44, units=pyunits.gal, bounds=(0, None)
)


@m.fs.cleaner_solex_settler.Constraint(CSX_mixer_settler_accounts)
def volume_scaling_constraint(c, k):
    return m.fs.cleaner_solex_settler.volume == pyunits.convert(
        REE_costing_params["1"][k]["RP Value"]
        * pyunits.gal
        * (
            m.fs.solex_cleaner_load.mscontactor.aqueous_inlet.flow_vol[0]
            / reference_basis_flow["cleaner_solex_aqueous_flow_vol"]
        ),
        to_units=pyunits.gal,
    )


m.fs.cleaner_solex_settler.costing = UnitModelCostingBlock(
    flowsheet_costing_block=m.fs.costing,
    costing_method=QGESSCostingData.get_REE_costing,
    costing_method_arguments={
        "cost_accounts": CSX_mixer_settler_accounts,
        "scaled_param": m.fs.cleaner_solex_settler.volume,
        "source": 1,
        "n_equip": 6,
        "scale_down_parallel_equip": False,
        "CE_index_year": CE_index_year,
    },
)

# Precipitation costs
# 10.1 is UKy Oxalate Precipitation - Polyethylene Tanks
reep_pe_tanks_accounts = ["10.1"]
m.fs.precipitator.volume = Var(initialize=15.04, units=pyunits.gal, bounds=(0, None))


@m.fs.precipitator.Constraint(reep_pe_tanks_accounts)
def volume_scaling_constraint(c, k):
    return m.fs.precipitator.volume == pyunits.convert(
        REE_costing_params["1"][k]["RP Value"]
        * pyunits.gal
        * (
            m.fs.precipitator.aqueous_inlet.flow_vol[0]
            / reference_basis_flow["precipitator_solex_aqueous_flow_vol"]
        ),
        to_units=pyunits.gal,
    )


m.fs.precipitator.costing = UnitModelCostingBlock(
    flowsheet_costing_block=m.fs.costing,
    costing_method=QGESSCostingData.get_REE_costing,
    costing_method_arguments={
        "cost_accounts": reep_pe_tanks_accounts,
        "scaled_param": m.fs.precipitator.volume,
        "source": 1,
        "n_equip": 1,
        "scale_down_parallel_equip": False,
        "CE_index_year": CE_index_year,
    },
)

# 10.2 is UKy Oxalate Precipitation - Tank Mixer
reep_tank_mixer_accounts = ["10.2"]
m.fs.precipitator_mixer = UnitModelBlock()
m.fs.precipitator_mixer.power = Var(initialize=0.61, units=pyunits.hp, bounds=(0, None))


@m.fs.precipitator_mixer.Constraint(reep_tank_mixer_accounts)
def power_scaling_constraint(c, k):
    return m.fs.precipitator_mixer.power == pyunits.convert(
        REE_costing_params["1"][k]["RP Value"]
        * pyunits.hp
        * (
            m.fs.precipitator.aqueous_inlet.flow_vol[0]
            / reference_basis_flow["precipitator_solex_aqueous_flow_vol"]
        ),
        to_units=pyunits.hp,
    )


m.fs.precipitator_mixer.costing = UnitModelCostingBlock(
    flowsheet_costing_block=m.fs.costing,
    costing_method=QGESSCostingData.get_REE_costing,
    costing_method_arguments={
        "cost_accounts": reep_tank_mixer_accounts,
        "scaled_param": m.fs.precipitator_mixer.power,
        "source": 1,
        "n_equip": 1,
        "scale_down_parallel_equip": False,
        "CE_index_year": CE_index_year,
    },
)

# 10.3 is UKy Oxalate Precipitation - Process Pump
reep_pump_accounts = ["10.3"]
m.fs.precipitator_pump = UnitModelBlock()
m.fs.precipitator_pump.costing = UnitModelCostingBlock(
    flowsheet_costing_block=m.fs.costing,
    costing_method=QGESSCostingData.get_REE_costing,
    costing_method_arguments={
        "cost_accounts": reep_pump_accounts,
        "scaled_param": m.fs.precipitator.aqueous_inlet.flow_vol[0],
        "source": 1,
        "n_equip": 1,
        "scale_down_parallel_equip": False,
        "CE_index_year": CE_index_year,
    },
)

# 10.4 is UKy Oxalate Precipitation - Filter Press
reep_filter_press_accounts = ["10.4"]
m.fs.sl_sep2.volume = Var(initialize=0.405, units=pyunits.ft**3, bounds=(0, None))


@m.fs.sl_sep2.Constraint(reep_filter_press_accounts)
def volume_scaling_constraint(c, k):
    return m.fs.sl_sep2.volume == pyunits.convert(
        REE_costing_params["1"][k]["RP Value"]
        * pyunits.ft**3
        * (
            m.fs.precipitator.aqueous_inlet.flow_vol[0]
            / reference_basis_flow["precipitator_solex_aqueous_flow_vol"]
        ),
        to_units=pyunits.ft**3,
    )


m.fs.sl_sep2.costing = UnitModelCostingBlock(
    flowsheet_costing_block=m.fs.costing,
    costing_method=QGESSCostingData.get_REE_costing,
    costing_method_arguments={
        "cost_accounts": reep_filter_press_accounts,
        "scaled_param": m.fs.sl_sep2.volume,
        "source": 1,
        "n_equip": 1,
        "scale_down_parallel_equip": False,
        "CE_index_year": CE_index_year,
    },
)

# 10.5 is UKy Oxalate Precipitation - Roaster
reep_roaster_accounts = ["10.5"]
m.fs.roaster.costing = UnitModelCostingBlock(
    flowsheet_costing_block=m.fs.costing,
    costing_method=QGESSCostingData.get_REE_costing,
    costing_method_arguments={
        "cost_accounts": reep_roaster_accounts,
        "scaled_param": abs(m.fs.roaster.heat_duty[0]),
        "source": 1,
        "n_equip": 1,
        "scale_down_parallel_equip": False,
        "CE_index_year": CE_index_year,
    },
)

# 3.3 Add Data Needed For Plantwide Costing

Before building plantwide costing (installation for equipment, fixed operating costs, variable operating costs, overnight costs), we need to add some more components to our model.

The framework calculates the sales revenue based on product composition and mass flows, so we need to define those as a dictionary:

In [6]:
# Molecular weights for convenience
REO_molar_mass = {
    "Y2O3": 88.906 * 2 + 16 * 3,
    "La2O3": 138.91 * 2 + 16 * 3,
    "Ce2O3": 140.12 * 2 + 16 * 3,
    "Pr2O3": 140.91 * 2 + 16 * 3,
    "Nd2O3": 144.24 * 2 + 16 * 3,
    "Sm2O3": 150.36 * 2 + 16 * 3,
    "Gd2O3": 157.25 * 2 + 16 * 3,
    "Dy2O3": 162.5 * 2 + 16 * 3,
    "Sc2O3": 44.96 * 2 + 16 * 3,
}


# The flowsheet state variable for flow is molar flow
# Components are all in the form X2O3 so we can do this compactly
# Add a function to create mass flow parameters
def product_mass_flow(blk, component):
    param = Param(
        default=pyunits.convert(
            m.fs.roaster.flow_mol_comp_product[0, component]
            * REO_molar_mass[component + "2O3"]
            * pyunits.g
            / pyunits.mol,
            to_units=pyunits.kg / pyunits.hr,
        ),
        units=pyunits.kg / pyunits.hr,
        mutable=True,
    )

    return param


# Create the dictionary
m.fs.Y_product = product_mass_flow(m.fs, "Y")
m.fs.La_product = product_mass_flow(m.fs, "La")
m.fs.Ce_product = product_mass_flow(m.fs, "Ce")
m.fs.Pr_product = product_mass_flow(m.fs, "Pr")
m.fs.Nd_product = product_mass_flow(m.fs, "Nd")
m.fs.Sm_product = product_mass_flow(m.fs, "Sm")
m.fs.Gd_product = product_mass_flow(m.fs, "Gd")
m.fs.Dy_product = product_mass_flow(m.fs, "Dy")
m.fs.Sc_product = product_mass_flow(m.fs, "Sc")

pure_product_output_rates = {}

mixed_product_output_rates = {
    "CeO2": m.fs.Ce_product,
    "Sc2O3": m.fs.Sc_product,
    "Y2O3": m.fs.Y_product,
    "La2O3": m.fs.La_product,
    "Nd2O3": m.fs.Nd_product,
    "Pr6O11": m.fs.Pr_product,
    "Sm2O3": m.fs.Sm_product,
    "Gd2O3": m.fs.Gd_product,
    "Dy2O3": m.fs.Dy_product,
}

The framework considers pure products separately from mixed products, which are products that exist in a "mixed basket" with many other products. These mixed baskets may be sold at a reduced price realization rather than discarded or further purified. In our case, all oxides are part of the mixed basket and we have no pure product streams.

Next, we need to define the streams for the variable operating costs. Let's add the power to operate the tank mixers and the disposal cost of the leach filter waste:

In [7]:
# Power for tank mixers
m.fs.power = Var(m.fs.time, initialize=7, units=pyunits.hp)
m.fs.power_constraint = Constraint(
    expr=m.fs.power[0]
    == pyunits.convert(
        m.fs.precipitator_mixer.power
        + m.fs.cleaner_mixer.power
        + m.fs.rougher_mixer.power
        + m.fs.leach_mixer.power,
        to_units=pyunits.hp,
    )
)

# Solid waste from leach filter
m.fs.solid_waste = Var(m.fs.time, initialize=0.0245, units=pyunits.ton / pyunits.hr)
m.fs.solid_waste_constraint = Constraint(
    expr=m.fs.solid_waste[0]
    == pyunits.convert(
        m.fs.leach_filter_cake.flow_mass[0], to_units=pyunits.ton / pyunits.hr
    )
)

Finally, let's add the land cost; this will be included in the overnight cost calculation. The land leasing cost is taken as $0.303736 per ton of feed processed per day, which is the total annual cost of lease agreements normalized by the total annual feedstock rate to the plant as presented by the [University of Kentucky report](https://doi.org/10.2172/1569277).

We'll also define an annual REE recovery rate so the method can calculate the REE recovery cost:

In [8]:
# Some time quantities for convenience
hours_per_shift = 8 * pyunits.hr
shifts_per_day = 3 * pyunits.day**-1
operating_days_per_year = 336 * pyunits.day
m.fs.annual_operating_hours = Param(
    initialize=hours_per_shift * shifts_per_day * operating_days_per_year,
    mutable=True,
    units=pyunits.hours / pyunits.a,
)

# Land leasing cost
m.fs.land_cost = Expression(
    expr=0.303736
    * 1e-6
    * getattr(pyunits, "MUSD_" + CE_index_year)
    / pyunits.ton  # leasing cost in flowsheet cost year basis
    * pyunits.convert(
        m.fs.leach_solid_feed.flow_mass[0], to_units=pyunits.ton / pyunits.hr
    )
    * m.fs.annual_operating_hours
    * pyunits.a
)

# Annual recovery rate
m.fs.recovery_rate_per_year = Var(initialize=13.306, units=pyunits.kg / pyunits.yr)
m.fs.recovery_rate_per_year_constraint = Constraint(
    expr=m.fs.recovery_rate_per_year
    == pyunits.convert(
        m.fs.roaster.flow_mass_product[0] * m.fs.annual_operating_hours,
        to_units=pyunits.kg / pyunits.yr,
    )
)

# 3.4 Build Plantwide Costs

Now, let's build the plantwide costs. This includes installation costs (Lang factor), overnight costs, and fixed and variable operating costs. The values set below are from the [University of Kentucky report](https://doi.org/10.2172/1569277) and are thus case-specific.

We can do this with one method call:

In [9]:
m.fs.costing.build_process_costs(
    # arguments related to installation costs
    Lang_factor=2.97,
    # argument related to fixed O&M costs
    fixed_OM=True,  # calculate fixed O&M costs
    labor_types=[
        "skilled",
        "unskilled",
        "supervisor",
        "maintenance",
        "technician",
        "engineer",
    ],
    labor_rate=[24.98, 19.08, 30.39, 22.73, 21.97, 45.85],  # USD/hr
    labor_burden=25,  # % fringe benefits
    operators_per_shift=[4, 9, 2, 2, 2, 3],
    hours_per_shift=hours_per_shift,
    shifts_per_day=shifts_per_day,
    operating_days_per_year=operating_days_per_year,
    pure_product_output_rates=pure_product_output_rates,
    mixed_product_output_rates=mixed_product_output_rates,
    mixed_product_sale_price_realization_factor=0.65,  # 65% price realization for mixed products
    # arguments related to total owners costs
    land_cost=m.fs.land_cost,
    # arguments related to variable O&M costs
    variable_OM=True,  # calculate variable O&M costs
    resources=[
        "nonhazardous_solid_waste",
        "power",
    ],  # variable cost names
    rates=[
        m.fs.solid_waste,
        m.fs.power,
    ],  # variable cost rates
    efficiency=0.80,  # power usage efficiency, or fixed motor/distribution efficiency
    waste=[
        "nonhazardous_solid_waste",
    ],  # waste is considered for the overnight costs
    recovery_rate_per_year=m.fs.recovery_rate_per_year,
    CE_index_year=CE_index_year,
)

# 3.5 Add Additional Costs

Suppose we have an additional cost we want to include, such as initial fills of tank chemicals. We can use a variable create by the `build_process_costs()` method to do so. Note that these quantities are also scaled from the [University of Kentucky report](https://doi.org/10.2172/1569277) by their respective section flowrates using an exponent of 0.7, which is a common selection for units which scale by volume or volumetric flowrate:

In [10]:
# Define reagent fill costs as an other plant cost so framework adds this to TPC calculation
m.fs.costing.other_plant_costs.unfix()
m.fs.costing.other_plant_costs_eq = Constraint(
    expr=(
        m.fs.costing.other_plant_costs
        == pyunits.convert(
            1218.073
            * pyunits.USD_2016  # Rougher Solvent Extraction
            * (
                m.fs.solex_rougher_load.mscontactor.aqueous_inlet.flow_vol[0]
                / reference_basis_flow["rougher_solex_aqueous_flow_vol"]
            )
            ** 0.7
            + 48.723
            * pyunits.USD_2016  # Cleaner Solvent Extraction
            * (
                m.fs.solex_cleaner_load.mscontactor.aqueous_inlet.flow_vol[0]
                / reference_basis_flow["cleaner_solex_aqueous_flow_vol"]
            )
            ** 0.7
            + 182.711
            * pyunits.USD_2016  # Solvent Extraction Wash and Saponification
            * (
                m.fs.precipitator.aqueous_inlet.flow_vol[0]
                / reference_basis_flow["precipitator_solex_aqueous_flow_vol"]
            )
            ** 0.7,
            to_units=getattr(pyunits, "MUSD_" + CE_index_year),
        )
    )
)

# 4 Solving and Displaying Results

# 4.1 Initialization and Solving

It is always a good idea to initialize models by pre-calculating some variables before attempting to solve the entire model. This helps the solver start closer to the final solution and limits the risk of divergence.

The `QGESSCosting()` class has some built-in methods to initialize the costing equations:

In [11]:
QGESSCostingData.costing_initialization(m.fs.costing)
QGESSCostingData.initialize_fixed_OM_costs(m.fs.costing)
QGESSCostingData.initialize_variable_OM_costs(m.fs.costing)

Now, let's solve the model:

In [12]:
solver = get_solver("ipopt_v2")
results = solver.solve(m, tee=True)
assert_optimal_termination(results)

Ipopt 3.13.2: linear_solver="ma57"
max_iter=200
nlp_scaling_method="gradient-based"
tol=1e-06


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collect

# 4.2 Reporting Cost Results
Let's call some built-in reporting method to view the cost results:

In [13]:
QGESSCostingData.report(m.fs.costing)
# TODO validate these results
# assert m.fs.costing.total_BEC.value == pytest.approx(0.52043, rel=1e-4)
# assert m.fs.costing.total_installation_cost.value == pytest.approx(1.0252, rel=1e-4)
# assert m.fs.costing.total_plant_cost.value == pytest.approx(1.5457, rel=1e-4)
# assert m.fs.costing.total_fixed_OM_cost.value == pytest.approx(6.8275, rel=1e-4)
# assert m.fs.costing.total_sales_revenue.value == pytest.approx(0, abs=1e-4)
# assert m.fs.costing.total_variable_OM_cost[0].value == pytest.approx(1.3656, rel=1e-4)
# assert m.fs.costing.plant_overhead_cost[0].value == pytest.approx(1.3655, rel=1e-4)
# assert value(m.fs.costing.cost_of_recovery) == pytest.approx(4.7736e07, rel=1e-4)


costing
------------------------------------------------------------------------------------
                                                                           Value   
    Plant Cost Units                                                      MUSD_2023
    Total Plant Cost                                                         1.5481
    Total Bare Erected Cost                                                 0.52124
    Total Installation Cost                                                  1.0268
    Total Other Plant Costs                                              9.9460e-06
    Total Fixed Operating & Maintenance Cost                                 6.8276
    Total Annual Operating Labor Cost                                        3.8090
    Total Annual Technical Labor Cost                                        1.8294
    Summation of Annual Labor Costs                                          5.6384
    Total Maintenance and Material Cost                           

In [14]:
m.fs.costing.variable_operating_costs.display()

variable_operating_costs : Variable operating costs
    Size=2, Index=fs._time*{nonhazardous_solid_waste, power}, Units=MUSD_2023/a
    Key                               : Lower : Value                  : Upper : Fixed : Stale : Domain
    (0.0, 'nonhazardous_solid_waste') :  None :  2.596407709179655e-10 :  None : False : False :  Reals
                       (0.0, 'power') :  None : 2.1070262144170174e-05 :  None : False : False :  Reals


In [15]:
QGESSCostingData.display_bare_erected_costs(m.fs.costing)

-----Bare Erected Costs (MUSD)-----
fs.leach.costing: 0.00048
fs.sl_sep1.costing: 0.00258
fs.leach_mixer.costing: 0.00568
fs.rougher_mixer.costing: 0.00220
fs.cleaner_mixer.costing: 0.00036
fs.leach_sx_mixer.costing: 0.00049
fs.precipitator.costing: 0.00022
fs.sl_sep2.costing: 0.00022
fs.roaster.costing: 0.04833
fs.leach_pump.costing: 0.01893
fs.leach_solution_heater.costing: 0.00022
fs.rougher_solex_tank.costing: 0.00029
fs.rougher_pump.costing: 0.00926
fs.rougher_solex_settler.costing: 0.35358
fs.cleaner_solex_tank.costing: 0.00022
fs.cleaner_pump.costing: 0.00692
fs.cleaner_solex_settler.costing: 0.06897
fs.precipitator_mixer.costing: 0.00047
fs.precipitator_pump.costing: 0.00185


In [16]:
QGESSCostingData.display_flowsheet_cost(m.fs.costing)



Total bare erected cost (MUSD): 0.521
Total overnight (installed) equipment cost: 1.548
Total annualized capital cost (MUSD): 0.177

Total annual fixed O&M cost (MUSD): 6.828
Total annual variable O&M cost (MUSD): 1.366
Total annual O&M cost (MUSD): 8.193
Total annual O&M cost per kg REE recovered (USD/kg): 48771318.341

Total annualized plant cost (MUSD): 8.371
Annual rate of recovery (kg/year): 0.168
Cost of recovery per kg REE recovered (USD/kg): 49827654.233



